In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow.sklearn
import mlflow
from mlflow.models.signature import infer_signature
from mlflow.utils.mlflow_tags import MLFLOW_GIT_COMMIT, MLFLOW_SOURCE_TYPE, MLFLOW_GIT_BRANCH, MLFLOW_GIT_REPO_URL, MLFLOW_SOURCE_NAME

In [2]:
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")

In [4]:
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    print(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

In [5]:
#  Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [6]:
alpha = 0.645225
l1_ratio = 0.225345
# Hi From Docker

In [11]:
mlflow.set_experiment("Demo")   
import git 
repo = git.Repo("")
with mlflow.start_run(log_system_metrics=True) as run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)
    signature = infer_signature(train_x, lr.predict(train_x))

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    tag = {MLFLOW_GIT_COMMIT: repo.head.commit, MLFLOW_SOURCE_TYPE:'PROJECT', 
           MLFLOW_GIT_BRANCH:repo.active_branch, MLFLOW_SOURCE_NAME:repo.remote().url}
    mlflow.set_tags(tag)

    ##print(mlflow.get_tracking_uri())
    
    model_name = "ElasticnetWineModel"
    mlflow.sklearn.log_model(lr, "model", registered_model_name="Arpit", signature=signature)

2024/05/14 09:59:52 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/05/14 09:59:52 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Elasticnet model (alpha=0.645345, l1_ratio=0.835345):
  RMSE: 0.8193751892310815
  MAE: 0.6627844727428825
  R2: -0.025980058418293206


Registered model 'Arpit' already exists. Creating a new version of this model...
2024/05/14 09:59:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Arpit, version 6
Created version '6' of model 'Arpit'.
2024/05/14 09:59:58 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/14 09:59:58 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
